In [1]:
num_nodes = 50
margin = 10 # time margin for arrival before appointments
percentage_of_appointments = 0
GlobalSpanCostCoefficient = 0 # adjust
slack = 200000 # adjust
min_work_days = 7 # minimum number of work days per location

work_schedule = {
    1: [8 * 60, 19 * 60],
    2: [8 * 60, 19 * 60],
    3: [8 * 60, 19 * 60],
    4: [8 * 60, 19 * 60],
    5: [8 * 60, 18 * 60],
    6: [8 * 60, 18 * 60],
    7: [8 * 60, 18 * 60]
}
# week (mon-sun) constraints
max_days_off = 0
days_off = {5,6,7,4} # default Sunday no work
max_overnight_stays = 2
no_overnight_stays = {}
if len(no_overnight_stays) > 0:
    possible_overnight_stays = set(range(1, 8)) - no_overnight_stays
else:
    possible_overnight_stays = set(range(1, 8))
max_short_days = 0
short_days = {day for day, hours in work_schedule.items() if (hours[1] - hours[0]) / 60 <= 4}

# Check for short days or overnight stays defined on days off
removed_days = [day for day in list(days_off) if day in short_days]
for day in removed_days:
    days_off.remove(day)
    print(f"Removed day {day} from days off because it's a short day or overnight stay.")

# Check for more days off defined than max days off
if len(days_off) > max_days_off:
    max_days_off = len(days_off)
    print(f"Adjusted max days off to {max_days_off} because {len(days_off)} days off were defined.")

# Check for more short days defined than max short days
if len(short_days) > max_short_days:
    max_short_days = len(short_days)
    print(f"Adjusted max short days to {max_short_days} because {len(short_days)} short days were defined.")

Adjusted max days off to 4 because 4 days off were defined.


In [2]:
penalty_factor = [10, 10000, 300]
time_limit = 40
overnight_cost = 2
from src.routing import solve, create_data
nodes_df, time_matrix = create_data(min_work_days, days_off, percentage_of_appointments, num_nodes, penalty_factor)
options_df, figs, route_lists = solve(nodes_df, time_matrix, slack, days_off, max_days_off, short_days, max_short_days, no_overnight_stays, max_overnight_stays, work_schedule, margin, time_limit, overnight_cost, lunch_duration=30, first_optimization_algorithm='BEST_INSERTION', second_optimization_algorithm='GUIDED_LOCAL_SEARCH', GlobalSpanCostCoefficient=GlobalSpanCostCoefficient, verbose=False, visual=True, restrictive=False)

Option 0 with objective value 18300 and 29 dropped nodes


Option 1 with objective value 18740 and 28 dropped nodes


Option 2 with objective value 18690 and 28 dropped nodes


Option 3 with objective value 19382 and 28 dropped nodes


In [3]:
options_df

,obj_value,n_overnight_trips,nodes_dropped,n_fixed_appointments_cons,n_fixed_appointments,route_lists,gaps,Sum,Length,num_nodes_considered,overnight_days,short_days,n_short_days,off_days,n_days_off,blocks,respected_nodes,fixed_appointments_nodes
0,18300.0,0,57.99999999999999%,0.0,0.0,"{'1_day_trip_0': [(0, '11:00'), (13, '11:18'),...","[1, 1, 1, 0, 0, 0, 0]",3.0,7,21.0,[],[],0,"[4, 5, 6, 7]",4,{1: 3},"{0, 1, 5, 8, 10, 11, 13, 18, 22, 26, 27, 29, 3...",[]
1,18740.0,1,56.00000000000001%,0.0,0.0,"{'1_day_trip_0': [(0, '11:00'), (18, '11:14'),...","[1, 2, 0, 0, 0, 0]",3.0,6,22.0,[2],[],0,"[4, 5, 6, 7]",4,"{1: 1, 2: 1}","{0, 1, 2, 5, 8, 9, 10, 11, 13, 18, 22, 27, 29,...",[]
2,18690.0,1,56.00000000000001%,0.0,0.0,"{'1_day_trip_0': [(0, '11:05'), (19, '12:30'),...","[2, 1, 0, 0, 0, 0]",3.0,6,22.0,[1],[],0,"[4, 5, 6, 7]",4,"{2: 1, 1: 1}","{0, 1, 5, 8, 10, 11, 13, 18, 19, 21, 22, 26, 2...",[]
3,19382.0,2,56.00000000000001%,0.0,0.0,"{'3_day_trip_0': [(0, '11:43'), (13, '12:31'),...","[3, 0, 0, 0, 0]",3.0,5,22.0,"[1, 2]",[],0,"[4, 5, 6, 7]",4,{3: 1},"{0, 1, 2, 5, 8, 10, 11, 13, 18, 19, 21, 22, 26...",[]
